In [293]:
import pandas as pd
import numpy as np
import datetime as dt
import pathlib
import wtfutil

In [294]:
PATH_PREFIX=dt.datetime.utcnow().strftime('data/%Y/%m/%d/')
txpool_mpool = pd.concat(wtfutil.pqload_lastn(PATH_PREFIX, 'full', 1).values())

In [295]:
df_feerate = txpool_mpool.assign(feerate=txpool_mpool.satoshi/txpool_mpool.vsize).sort_values('feerate')

In [296]:
df = df_feerate.assign(rcs_vsize = df_feerate.vsize[::-1].cumsum())

In [297]:
%%time
[df.query("rcs_vsize > 72e6").tail(1).feerate]

CPU times: user 4 ms, sys: 0 ns, total: 4 ms
Wall time: 3.78 ms


[24713    1.003134
 Name: feerate, dtype: float64]

In [298]:
def mempool_ntiles(df_in, n=10, top=72, v=1e6):
    positions = [top/n*i + top/n/2 for i in range(n)]
    data = []
    for p in positions:
        t = df_in.query(f"rcs_vsize > {p * v}").tail(1)
        if len(t):
            data.append(float(t.feerate))
        else:
            data.append(None)
        
    return pd.Series(data, index=positions)

s = mempool_ntiles(df,50)


In [299]:
s= mempool_ntiles(df,9)
np.exp(np.round(np.log(s)*20)/20)

4.0     4.953032
12.0    4.263115
20.0    3.158193
28.0    2.225541
36.0    1.648721
44.0    1.349859
52.0    1.284025
60.0    1.000000
68.0    1.000000
dtype: float64

In [300]:
s= mempool_ntiles(df,9,9)
np.exp(np.round(np.log(s)*20)/20)

0.5    4.953032
1.5    4.953032
2.5    4.953032
3.5    4.953032
4.5    4.953032
5.5    4.953032
6.5    4.953032
7.5    4.953032
8.5    4.953032
dtype: float64

In [301]:
s= mempool_ntiles(df,10)
np.exp(np.round(np.log(s)*20)/20)

3.6     4.953032
10.8    4.953032
18.0    3.320117
25.2    2.585710
32.4    1.915541
39.6    1.419068
46.8    1.284025
54.0    1.161834
61.2    1.000000
68.4    1.000000
dtype: float64